In [1]:
import sys
import os

# Get the current notebook's directory
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
# Get the project root (two levels up from the notebook)
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
# Add the project root to Python path
sys.path.append(project_root)
print(f"Project root: {project_root}")

# Questrade Symbol Management Notebook

This notebook demonstrates how to use the `local_symbols.py` module to manage stock symbol data locally. The module provides functions for retrieving, storing, and working with stock symbol information from the Questrade API.

## Setup

In [2]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from pathlib import Path
import json

In [3]:
# Import the local_symbols module
import src.local_symbols as local_symbols

In [4]:
# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Exploring the Local Symbols Database

Let's first check if we have any symbols stored in our local database.


In [ ]:
# Check current database contents
symbols = local_symbols.get_all_local_symbols()
print(f"Found {len(symbols)} symbols in the local database")

if len(symbols) > 0:
    # Display a sample of the symbols
    symbols_df = pd.DataFrame([{
        'Symbol': s['symbol'],
        'Company': s['description'],
        'Exchange': s['listingExchange'],
        'Security Type': s['securityType'],
        'Industry': s['industrySector']
    } for s in symbols])
    
    display(symbols_df.head(10))

## Saving a New Symbol

Let's fetch and save data for a new symbol. This demonstrates how the module retrieves symbol details from Questrade and stores them locally.

In [ ]:
# Define a list of symbols to save
symbols_to_save = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

# Save each symbol
for symbol in symbols_to_save:
    try:
        symbol_data = local_symbols.save_local_symbol(symbol)
        print(f"Successfully saved {symbol} - {symbol_data['description']}")
    except Exception as e:
        print(f"Error saving {symbol}: {e}")

## Retrieving Symbol Information

Now let's demonstrate how to retrieve symbol information from the local database.


In [ ]:
# Retrieve a symbol
symbol_to_retrieve = 'AAPL'
symbol_data = local_symbols.get_local_symbol(symbol_to_retrieve)

# Convert to DataFrame for better display
symbol_df = pd.DataFrame([symbol_data])

# Display basic information
print(f"Symbol: {symbol_data['symbol']}")
print(f"Company: {symbol_data['description']}")
print(f"Exchange: {symbol_data['listingExchange']}")
print(f"Industry: {symbol_data['industryGroup']}")
print(f"Security Type: {symbol_data['securityType']}")
print(f"Currency: {symbol_data['currency']}")
print(f"Prev Close: ${symbol_data['prevDayClosePrice']:.2f}")
print(f"52-Week High: ${symbol_data['highPrice52']:.2f}")
print(f"52-Week Low: ${symbol_data['lowPrice52']:.2f}")
print(f"Average Volume (3M): {symbol_data['averageVol3Months']:,}")
print(f"Market Cap: ${symbol_data['marketCap']:,}")
print(f"EPS: ${symbol_data['eps']:.2f}")
if symbol_data['pe'] is not None:
    print(f"P/E Ratio: {symbol_data['pe']:.2f}")
print(f"Dividend: ${symbol_data['dividend']:.2f}")
print(f"Dividend Yield: {symbol_data['yield']:.2f}%")
print(f"Has Options: {symbol_data['hasOptions']}")

Let's visualize some of the data to better understand the symbol's characteristics:


In [ ]:
# Create a figure with subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Price range visualization
price_data = {
    '52-Week Low': symbol_data['lowPrice52'],
    'Previous Close': symbol_data['prevDayClosePrice'],
    '52-Week High': symbol_data['highPrice52']
}
prices = list(price_data.values())
labels = list(price_data.keys())

# Bar chart for price range
ax1.bar(labels, prices, color=['blue', 'green', 'orange'])
ax1.set_title(f"{symbol_data['symbol']} - Price Range")
ax1.set_ylabel("Price ($)")
# Add price labels on top of bars
for i, price in enumerate(prices):
    ax1.text(i, price + 2, f"${price:.2f}", ha='center')

# Pie chart for industry categorization
industry_data = {
    'Sector': symbol_data['industrySector'],
    'Group': symbol_data['industryGroup'],
    'Subgroup': symbol_data['industrySubgroup']
}
ax2.axis('equal')
ax2.pie([1], labels=[industry_data['Sector']], autopct='%1.1f%%', 
       startangle=90, colors=['skyblue'])
ax2.set_title(f"{symbol_data['symbol']} - Industry Classification")

plt.tight_layout()
plt.show()

# Display more details about ticks and trading information
print("\nTrading Information:")
print("-" * 30)
print(f"Min Ticks:")
for tick in symbol_data['minTicks']:
    print(f"  - Price: ${tick.get('price', 'N/A')}, Value: ${tick.get('value', 'N/A')}")


## Forcing Data Update

The `get_local_symbol` function has a `force_update` parameter that allows us to refresh the data from the Questrade API.


In [ ]:
# Get the latest data from the API
updated_symbol_data = local_symbols.get_local_symbol('AAPL', force_update=True)

# Check what fields might have changed
print("Updated Symbol Data:")
print(f"Previous Day Close: ${updated_symbol_data['prevDayClosePrice']:.2f}")
print(f"Market Cap: ${updated_symbol_data['marketCap']:,}")

# Display the timestamp of the last update
import sqlite3
from pathlib import Path

db_path = Path('../data/symbols.db')
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute('SELECT last_updated FROM symbols WHERE symbol = ?', ('AAPL',))
last_updated = cursor.fetchone()[0]
conn.close()

print(f"Last Updated: {last_updated}")


## Importing Symbols from a CSV File

The module provides a function to import multiple symbols from a CSV file. Let's demonstrate this with a sample CSV file. First, we'll create a sample CSV file with some symbols:


In [ ]:
# Create a sample CSV file with some Dow Jones Industrial Average symbols
csv_content = """Company,Symbol
Apple Inc.,AAPL
Microsoft Corporation,MSFT
Amazon.com Inc.,AMZN
Alphabet Inc.,GOOGL
Tesla Inc.,TSLA
Meta Platforms Inc.,META
Nvidia Corporation,NVDA
JP Morgan Chase & Co.,JPM
Johnson & Johnson,JNJ
Visa Inc.,V
"""

csv_path = "../lists/sample_symbols.csv"
os.makedirs(os.path.dirname(csv_path), exist_ok=True)
with open(csv_path, "w") as f:
    f.write(csv_content)

print(f"Created sample CSV file at {csv_path}")

Now let's import the symbols from this file:


In [ ]:
# Import symbols from the CSV file
imported_count = local_symbols.import_symbols_from_csv(csv_path)
print(f"Successfully imported {imported_count} symbols")

# Verify the imported symbols
all_symbols = local_symbols.get_all_local_symbols()
print(f"Total symbols in database: {len(all_symbols)}")

# Display all symbols in the database
all_symbols_df = pd.DataFrame([{
    'Symbol': s['symbol'],
    'Company': s['description'],
    'Exchange': s['listingExchange'],
    'Security Type': s['securityType'],
    'Industry': s.get('industrySector', 'N/A')
} for s in all_symbols])

display(all_symbols_df)

## Advanced Symbol Analysis

Now that we have several symbols in our database, let's perform some analysis on them.


In [ ]:
# Get all symbols from the database
symbols = local_symbols.get_all_local_symbols()

# Create a DataFrame with key metrics
metrics_df = pd.DataFrame([{
    'Symbol': s['symbol'],
    'Company': s['description'],
    'Price': s['prevDayClosePrice'],
    'Market Cap (B)': s['marketCap'] / 1_000_000_000 if s['marketCap'] else None,
    'P/E': s['pe'],
    'EPS': s['eps'],
    'Dividend Yield': s['yield'],
    '52W High': s['highPrice52'],
    '52W Low': s['lowPrice52'],
    'Volume (3M)': s['averageVol3Months'],
    'Industry': s['industrySector'] if 'industrySector' in s else 'N/A',
    'Exchange': s['listingExchange']
} for s in symbols])

# Sort by market cap
metrics_df = metrics_df.sort_values('Market Cap (B)', ascending=False)

# Display the metrics
display(metrics_df)

# Create a visualization of market caps
plt.figure(figsize=(12, 8))
bar_plot = metrics_df.head(10).plot(
    kind='bar', 
    x='Symbol', 
    y='Market Cap (B)', 
    color='skyblue', 
    alpha=0.7,
    title='Top 10 Companies by Market Cap',
    ax=plt.gca()
)
plt.ylabel('Market Cap (Billions $)')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add value labels on top of bars
for i, v in enumerate(metrics_df.head(10)['Market Cap (B)']):
    plt.text(i, v + 5, f"${v:.1f}B", ha='center')

plt.tight_layout()
plt.show()

# Plot P/E ratio comparison
valid_pe = metrics_df[metrics_df['P/E'].notna()].sort_values('P/E')
if not valid_pe.empty:
    plt.figure(figsize=(12, 8))
    bar_plot = valid_pe.head(10).plot(
        kind='bar', 
        x='Symbol', 
        y='P/E', 
        color='lightgreen', 
        alpha=0.7,
        title='Companies by P/E Ratio (Lowest to Highest)',
        ax=plt.gca()
    )
    plt.ylabel('P/E Ratio')
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Add value labels on top of bars
    for i, v in enumerate(valid_pe.head(10)['P/E']):
        if not pd.isna(v):
            plt.text(i, v + 0.5, f"{v:.1f}", ha='center')

    plt.tight_layout()
    plt.show()


## Industry Sector Analysis

Let's analyze the distribution of industry sectors in our dataset.


In [ ]:
symbols

In [ ]:
# Check if we have industry data
if 'Industry' in metrics_df.columns:
    # Create industry analysis
    industry_counts = metrics_df['Industry'].value_counts()
    
    # Plot industry distribution
    plt.figure(figsize=(12, 8))
    industry_counts.plot(
        kind='pie',
        autopct='%1.1f%%',
        startangle=90,
        shadow=False,
        title='Distribution by Industry Sector'
    )
    plt.axis('equal')
    plt.tight_layout()
    plt.show()
    
    # Show average metrics by industry
    industry_metrics = metrics_df.groupby('Industry').agg({
        'Price': 'mean',
        'Market Cap (B)': 'mean',
        'P/E': 'mean',
        'Dividend Yield': 'mean',
        'Symbol': 'count'
    }).rename(columns={'Symbol': 'Count'}).sort_values('Market Cap (B)', ascending=False)
    
    display(industry_metrics)


## Searching for Symbol Information

Let's create a simple function to search for symbols in our database.


In [ ]:
def search_symbols(keyword):
    """
    Search for symbols in the local database that match the keyword.
    
    Args:
        keyword (str): Keyword to search for in symbol or description
        
    Returns:
        pd.DataFrame: Matching symbols
    """
    all_symbols = local_symbols.get_all_local_symbols()
    matching_symbols = []
    
    for s in all_symbols:
        if (keyword.upper() in s['symbol'].upper() or 
            keyword.upper() in s['description'].upper()):
            matching_symbols.append({
                'Symbol': s['symbol'],
                'Company': s['description'],
                'Price': s['prevDayClosePrice'],
                'Market Cap (B)': s['marketCap'] / 1_000_000_000 if s['marketCap'] else None,
                'Industry': s.get('industrySector', 'N/A'),
                'Exchange': s['listingExchange']
            })
    
    return pd.DataFrame(matching_symbols)

# Example searches
search_terms = ['TECH', 'BANK', 'AUTO']

for term in search_terms:
    results = search_symbols(term)
    if not results.empty:
        print(f"\nResults for '{term}':")
        display(results)
    else:
        print(f"\nNo results found for '{term}'")


## Conclusion

In this notebook, we've explored the functionality of the `local_symbols.py` module, which allows us to:

1. Save symbol data from Questrade to a local SQLite database
2. Retrieve symbol information from the local database
3. Force updates of symbol data when needed
4. Import multiple symbols from a CSV file
5. Perform analysis on the symbols in our database

This local database approach provides several advantages:
- Reduces API calls to Questrade's servers
- Provides faster access to symbol information
- Allows for offline analysis of symbol data
- Enables batch processing of multiple symbols

The module is particularly useful for applications that need to repeatedly access symbol information, such as portfolio analysis tools, screeners, or algorithmic trading systems.